### Your lab resolution :

In [37]:
import requests

# The API stuff
# I removed my token and other bits that were preventing a push for security etc.

# This is our cohort: ta-data-pt-rmt but I keep getting 404, is the repo not public or am I using the wrong end point?

# I'm going to try my own repo to test:

url = 'https://api.github.com/users/alexg119/repos'

response = requests.get(url, headers=headers)
forks = response.json()

if response.status_code == 200:
    print("success")
else:
    print(response.status_code)



languages = set()  # a set for unique entries

for fork in forks:
    language = fork.get('language')
    if language:
        languages.add(language)

print(languages)

# This works. I'll try the Ironhack public repo again

success
{'Jupyter Notebook', 'Python'}


In [38]:
# org_url = 'https://api.github.com/orgs/github/repos' - this works.
org_url = 'https://api.github.com/orgs/ta-data-pt-rmt/repos'

response = requests.get(org_url, headers=headers)

if response.status_code == 200:
    repos = response.json()
    for repo in repos:
        print(repo['html_url'])  # i tried to print just repo but it was not easy to read, so I extracted just the URL instead
else:
    print(response.status_code)

# I'm stuck here because I don't know the end point for ta-data-pt-rmt
# Maybe I misunderstood the question and I'm supposed to use the 2019 example and not the actual bootcamp i am in?

404


In [23]:
repo_url = 'https://api.github.com/repos/ironhack-datalabs/bcn-feb-2019/forks'

response = requests.get(repo_url, headers=headers)

# Re-using code from cell 1
if response.status_code == 200:
    forks = response.json()
    languages = set()  # 

    for fork in forks:
        language = fork.get('language')
        if language:
            languages.add(language)

    print(list(languages))
else:
    print(response.status_code)


['Jupyter Notebook']


### I'm going to try challenge 2, I can't seem to figure out what I'm supposed to be doing for part 1 ###

In [55]:
from datetime import datetime, timedelta

# Let's figure out 3 weeks ago
three_weeks_ago = datetime.now() - timedelta(weeks=3)
since_date = three_weeks_ago.isoformat()

print(since_date) # works

# The lab we need:
owner = 'ta-data-pt-rmt'
repo = 'lab-mysql-first-queries'

#Let's add that to our url:
commits_url = "https://api.github.com/repos/"+owner+"/"+repo+"/commits"
#print(commits_url) it works

params = {'since': since_date}
response = requests.get(commits_url, headers=headers, params=params)

if response.status_code == 200:
    commits = response.json()
    commit_count = len(commits)
    print(commit_count)
else:
    print(response.status_code)

# So I checked and that's true, the last commit is 2 years ago here: https://github.com/ta-data-pt-rmt/lab-mysql-first-queries/commits/main/
# Should I be checking pull requests instead?

pulls_url = "https://api.github.com/repos/"+owner+"/"+repo+"/pulls"

params = {'since': since_date}
response = requests.get(pulls_url, headers=headers, params=params)

if response.status_code == 200:
    pull_requests = response.json()
    #display(response.json())
else:
    print(response.status_code)

# I'll use created here after looking at the .json

params = {
    'state': 'all',       
    'sort': 'created',  
}
recent_pulls = []

for pr in pull_requests:
    if pr['created_at'] >= since_date:
            recent_pulls.append(pr) 
#print(recent_pulls)

pull_count = len(recent_pulls)

print(pull_count) # Yay! works

2024-06-02T16:18:28.836892
0
2


## I struggled a bit with this lab (with the instructions), the API stuff was fine. I couldn't definitively understand the requirements, but I think I've got a hang on APIs ##